# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
from google.colab import auth
auth.authenticate_user() # Authenticate to Google Cloud

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # Set the region for consistency

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID # Export project ID as an environment variable

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project

print("Project:", PROJECT_ID, "| Region:", REGION)

# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-467-project-1
Updated property [core/project].
mgmt-467-project-1
Project: mgmt-467-project-1 | Region: us-central1


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [ ]:
!gcloud config get-value project
!echo "Region: $REGION"

mgmt-467-project-1
Region: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

We set them at the top to pull the correct resources and for reproducibility. If we don't, we may pull resources from other areas or create the wrong resources during our attempts to analyze data. It may also lead to inconsistent performance and mass errors.

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [ ]:
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
# Create directory for Kaggle config if it doesn't exist
os.makedirs('/root/.kaggle', exist_ok=True)
# Save the uploaded kaggle.json file
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
# Set file permissions to 0600 for security (owner read/write only)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Verify Kaggle installation
!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle (1).json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


In [ ]:
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

It's critical for for security and preventing unauthorized access to our files. We are avoiding risking other users gaining access to our account and malicious software from accessing our info.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [ ]:
# Create directory for raw data
!mkdir -p /content/data/raw

# Download the dataset using Kaggle CLI
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data

# Unzip the downloaded dataset into the raw data directory (overwrite existing files)
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
netflix-2025user-behavior-dataset-210k-records.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 1

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [ ]:
import glob
import os

csv_files = glob.glob('/content/data/raw/*.csv')

# Assert there are exactly six CSV files
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"

print("Found 6 CSV files:")
for csv_file in csv_files:
    print(os.path.basename(csv_file))

Found 6 CSV files:
movies.csv
watch_history.csv
search_logs.csv
users.csv
recommendation_logs.csv
reviews.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

It's important because it allows for efficient access and location when we require these files in the future for other projects or inspection. When we let lots of cluttered files build up, we may accidentally reference the wrong file or delete something important while mass cleaning.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [ ]:
import os, uuid

PROJECT_ID = ""
REGION = "us-central1"

bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name

print(f"Creating bucket {bucket_name} in region {REGION}...")
cmd = f"gcloud storage buckets create gs://{bucket_name} --location={REGION}"
if PROJECT_ID:
    cmd += f" --project {PROJECT_ID}"
!{cmd}

print(f"Uploading CSV files to gs://{bucket_name}/netflix/...")
!gcloud storage cp /content/data/raw/*.csv gs://{bucket_name}/netflix/

Creating bucket mgmt467-netflix-7784f2ec in region us-central1...
Creating gs://mgmt467-netflix-7784f2ec/...
Uploading CSV files to gs://mgmt467-netflix-7784f2ec/netflix/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-7784f2ec/netflix/movies.csv
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-7784f2ec/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-7784f2ec/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-7784f2ec/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-7784f2ec/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-7784f2ec/netflix/watch_history.csv

Average throughput: 9.5MiB/s


### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [ ]:
!gcloud storage ls -l gs://$BUCKET_NAME/netflix/

    115942  2025-10-26T23:04:44Z  gs://mgmt467-netflix-7784f2ec/netflix/movies.csv
   4695557  2025-10-26T23:04:45Z  gs://mgmt467-netflix-7784f2ec/netflix/recommendation_logs.csv
   1861942  2025-10-26T23:04:45Z  gs://mgmt467-netflix-7784f2ec/netflix/reviews.csv
   2250902  2025-10-26T23:04:45Z  gs://mgmt467-netflix-7784f2ec/netflix/search_logs.csv
   1606820  2025-10-26T23:04:45Z  gs://mgmt467-netflix-7784f2ec/netflix/users.csv
   9269425  2025-10-26T23:04:46Z  gs://mgmt467-netflix-7784f2ec/netflix/watch_history.csv
TOTAL: 6 objects, 19800588 bytes (18.88MiB)


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

One benefit is that it's extremely efficienct due to GCS being able to store massive amounts of data and load it optimally. Loading from local environments can be slow depending on resources and hardware. Another benefit is that it's very reliable because it's stored across multiple devices and sources so it can be accessed even during issues.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [ ]:
# Cell A: Create BigQuery dataset (idempotently)
DATASET = "netflix"
print(f"Creating dataset {DATASET} in US multi-region...")
# Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist or there was another issue."

Creating dataset netflix in US multi-region...
BigQuery error in mk operation: Dataset 'mgmt-467-project-1:netflix' already
exists.
Dataset may already exist or there was another issue.


In [ ]:
import os

DATASET = "netflix"
PROJECT = os.environ.get("GOOGLE_CLOUD_PROJECT") or os.environ.get("PROJECT_ID")

tables = {
    "users": "users.csv",
    "movies": "movies.csv",
    "watch_history": "watch_history.csv",
    "recommendation_logs": "recommendation_logs.csv",
    "search_logs": "search_logs.csv",
    "reviews": "reviews.csv",
}

# --- Load each table from GCS ---
for tbl, fname in tables.items():
    src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
    print(f"\nLoading {tbl} from {src}")
    # add --replace to overwrite instead of append if you need a fresh load
    !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# --- Row counts (one query per table) ---
print("\nRow counts for loaded tables:")
for tbl in tables.keys():
    # No backticks, no project prefix -> avoids shell command substitution issues
    sql_query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS row_count FROM {DATASET}.{tbl}"
    print("\nSQL:", sql_query)
    if PROJECT:
        !bq query --project_id {PROJECT} --nouse_legacy_sql "{sql_query}"
    else:
        !bq query --nouse_legacy_sql "{sql_query}"


Loading users from gs://mgmt467-netflix-7784f2ec/netflix/users.csv
Waiting on bqjob_r409ac212ccefd0cf_0000019a22c740ee_1 ... (1s) Current status: DONE   

Loading movies from gs://mgmt467-netflix-7784f2ec/netflix/movies.csv
Waiting on bqjob_r6b95902effbd4209_0000019a22c7647d_1 ... (0s) Current status: DONE   

Loading watch_history from gs://mgmt467-netflix-7784f2ec/netflix/watch_history.csv
Waiting on bqjob_r6c2b0be47ce85b67_0000019a22c78103_1 ... (1s) Current status: DONE   

Loading recommendation_logs from gs://mgmt467-netflix-7784f2ec/netflix/recommendation_logs.csv
Waiting on bqjob_r4b25ce885cf4925a_0000019a22c7a1b4_1 ... (1s) Current status: DONE   

Loading search_logs from gs://mgmt467-netflix-7784f2ec/netflix/search_logs.csv
Waiting on bqjob_r4507eb2e8c90f6d8_0000019a22c7c38c_1 ... (1s) Current status: DONE   

Loading reviews from gs://mgmt467-netflix-7784f2ec/netflix/reviews.csv
Waiting on bqjob_r32674f258fee8dbc_0000019a22c7e411_1 ... (1s) Current status: DONE   

Row cou

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [ ]:
%%bigquery --project mgmt-467-project-1
SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,reviews,77250
1,movies,6240
2,watch_history,630000
3,search_logs,132500
4,recommendation_logs,260000
5,users,61800


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

It's good during fast EDA and prototyping because it saves time and resoures. Enforcing explicit schemas is important when ensuring data quality and building data pipelines because the process needs to be very meticulous and reliable.

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 1: Total rows and % missing for specific columns in the users table
SELECT
  COUNT(*) AS total_rows,
  COUNTIF(country IS NULL) AS missing_country,
  ROUND(SAFE_DIVIDE(COUNTIF(country IS NULL), COUNT(*)) * 100, 2) AS pct_missing_country,
  COUNTIF(subscription_plan IS NULL) AS missing_subscription_plan,
  ROUND(SAFE_DIVIDE(COUNTIF(subscription_plan IS NULL), COUNT(*)) * 100, 2) AS pct_missing_subscription_plan,
  COUNTIF(age IS NULL) AS missing_age,
  ROUND(SAFE_DIVIDE(COUNTIF(age IS NULL), COUNT(*)) * 100, 2) AS pct_missing_age
FROM `mgmt-467-project-1.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,total_rows,missing_country,pct_missing_country,missing_subscription_plan,pct_missing_subscription_plan,missing_age,pct_missing_age
0,61800,0,0.0,0,0.0,7374,11.93


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 2: % missing by country
-- This helps identify if missingness of a key field is dependent on another variable (MAR)
SELECT
  country,
  COUNT(*) AS total_in_country,
  COUNTIF(subscription_plan IS NULL) AS missing_subscription_plan_in_country,
  ROUND(SAFE_DIVIDE(COUNTIF(subscription_plan IS NULL), COUNT(*)) * 100, 2) AS pct_missing_subscription_plan
FROM `mgmt-467-project-1.netflix.users`
GROUP BY country
ORDER BY pct_missing_subscription_plan DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,country,total_in_country,missing_subscription_plan_in_country,pct_missing_subscription_plan
0,USA,43224,0,0.0
1,Canada,18576,0,0.0


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Verification query: Print the three missingness percentages
SELECT
  ROUND(SAFE_DIVIDE(COUNTIF(country IS NULL), COUNT(*)) * 100, 2) AS pct_missing_country,
  ROUND(SAFE_DIVIDE(COUNTIF(subscription_plan IS NULL), COUNT(*)) * 100, 2) AS pct_missing_subscription_plan,
  ROUND(SAFE_DIVIDE(COUNTIF(age IS NULL), COUNT(*)) * 100, 2) AS pct_missing_age
FROM `mgmt-467-project-1.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,pct_missing_country,pct_missing_subscription_plan,pct_missing_age
0,0.0,0.0,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

Age is the columns missing the most values. MNAR is the most likely culprit due to individuals being careful of sharing their personal information and children accessing content that they would normally be age blocked out of.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [ ]:
# Show the schema of the watch_history table to identify the correct timestamp column
!bq show netflix.watch_history

Table mgmt-467-project-1:netflix.watch_history

   Last modified                 Schema                Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- ---------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  26 Oct 23:08:15   |- session_id: string              630000       58636872                                                          58636872              7061481                        
                    |- user_id: string                                                                                                                                                     
                    |- movie_id: string                                                                                                                                                    
            

In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 1: Report duplicate groups on (user_id, movie_id, watch_date, device_type)
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
FROM `mgmt-467-project-1.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,watch_date,device_type,dup_count
0,user_00391,movie_0893,2024-08-26,Laptop,24
1,user_03310,movie_0640,2024-09-08,Smart TV,24
2,user_09564,movie_0552,2025-01-11,Laptop,18
3,user_03094,movie_0114,2024-05-13,Smart TV,18
4,user_06462,movie_0588,2025-02-10,Laptop,18
5,user_06746,movie_0858,2024-01-23,Smart TV,18
6,user_09454,movie_0116,2025-10-19,Laptop,18
7,user_05080,movie_0714,2024-08-05,Smart TV,18
8,user_02822,movie_0009,2025-08-30,Desktop,18
9,user_03176,movie_0534,2024-01-06,Laptop,18


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 2: Create watch_history_dedup table keeping one row per group
-- Policy: prefer higher progress_percentage, then watch_duration_minutes
CREATE OR REPLACE TABLE `mgmt-467-project-1.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, watch_date, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt-467-project-1.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Verification query: Compare row counts before and after deduplication
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-project-1.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-project-1.netflix.watch_history_dedup`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history_dedup,100000
1,watch_history_raw,630000


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Natural duplicates occur when individuals perform actions or provide information to a system multiple times that falls within that system's designed data acceptance rules. System-generated occur when the data storage rules are designed with a quirk that allows multiple recordings of it without checking, resulting in duplicates. It can cause issues by inflating metrics such as users and user growth by providing major over estimations which can lead to poor strategy decisions.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 1: Compute IQR bounds for watch_duration_minutes on watch_history_dedup and report % outliers
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt-467-project-1.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt-467-project-1.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3482,100000,3.48


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 2: Create watch_history_robust table with watch_duration_minutes_capped capped at P01/P99; return quantile summaries before/after.
CREATE OR REPLACE TABLE `mgmt-467-project-1.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt-467-project-1.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `mgmt-467-project-1.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after (in a separate query if needed for display)
-- SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q FROM `mgmt-467-project-1.netflix.watch_history_dedup`
-- UNION ALL
-- SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q FROM `mgmt-467-project-1.netflix.watch_history_robust`;

Query is running:   0%|          |

""


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Verification query: Show min/median/max before vs after capping
SELECT
  'before_capping' AS stage,
  MIN(watch_duration_minutes) AS min_duration,
  APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_duration,
  MAX(watch_duration_minutes) AS max_duration
FROM `mgmt-467-project-1.netflix.watch_history_dedup`
UNION ALL
SELECT
  'after_capping' AS stage,
  MIN(watch_duration_minutes_capped) AS min_duration,
  APPROX_QUANTILES(watch_duration_minutes_capped, 2)[OFFSET(1)] AS median_duration,
  MAX(watch_duration_minutes_capped) AS max_duration
FROM `mgmt-467-project-1.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,stage,min_duration,median_duration,max_duration
0,before_capping,0.2,51.2,799.3
1,after_capping,4.4,51.4,204.0


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

It can be harmful if it distorts the original data. The extreme values can provide crucial insight and edge cases, but capping it cuts out the potential information gain. A model less sensitive to outliers would be decision trees.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 1: Compute and summarize flag_binge for sessions > 8 hours in watch_history_robust
SELECT
  COUNTIF(watch_duration_minutes_capped > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes_capped > 8*60)/COUNT(*),2) AS pct
FROM `mgmt-467-project-1.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct
0,0,100000,0.0


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 2: Compute and summarize flag_age_extreme if age < 10 or > 100 in users
SELECT
  COUNTIF(age < 10 OR age > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct
FROM `mgmt-467-project-1.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct
0,1074,61800,1.74


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Cell 3: Compute and summarize flag_duration_anomaly where duration_minutes < 15 or > 480 in movies
-- Using the corrected column name 'duration_minutes'
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 480) AS titles_over_8h,
  COUNT(*) AS total
FROM `mgmt-467-project-1.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,titles_under_15m,titles_over_8h,total
0,72,66,6240


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [ ]:
%%bigquery --project mgmt-467-project-1
-- Verification query: Single compact summary of anomaly flags
SELECT
  'flag_binge' AS flag_name,
  ROUND(100*COUNTIF(watch_duration_minutes_capped > 8*60)/COUNT(*),2) AS pct_of_rows
FROM `mgmt-467-project-1.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct_of_rows
FROM `mgmt-467-project-1.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly_under_15m' AS flag_name,
  ROUND(100*COUNTIF(duration_minutes < 15)/COUNT(*),2) AS pct_of_rows
FROM `mgmt-467-project-1.netflix.movies`
UNION ALL
SELECT
  'flag_duration_anomaly_over_8h' AS flag_name,
  ROUND(100*COUNTIF(duration_minutes > 480)/COUNT(*),2) AS pct_of_rows
FROM `mgmt-467-project-1.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.00
1,flag_age_extreme,1.74
2,flag_duration_anomaly_under_15m,1.15
3,flag_duration_anomaly_over_8h,1.06


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

The most common anomaly is flag_age_extreme because of the ammount of people under 10 years old and the ammount over 100 years old. These are likely typos or mistakes made in the system as this is extremely uncommon. Keeping the watch duration flags in is fine because some people buy a subscription and barely watch or use it and others may binge long sessions every day while at work or sitting at home.

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## 6) Save & submit checklist

- [ ] Save this notebook to your team's shared Google Drive folder.
- [ ] Export your BigQuery SQL queries used for data quality analysis into a single `.sql` file. Save this file locally.
- [ ] Clone your team's GitHub repository if you haven't already.
- [ ] Add both the saved notebook file (`.ipynb`) and the exported SQL file (`.sql`) to your local repository.
- [ ] Commit your changes with a descriptive message (e.g., "Completed DQ analysis and added notebook/SQL").
- [ ] Push your changes to the team GitHub repository.
- [ ] Create a README file in your repository (or update an existing one) that includes:
    - Your GCP `PROJECT_ID`
    - Your chosen `REGION`
    - The name of the GCS bucket created
    - The name of the BigQuery dataset created (`netflix`)
    - Today's row counts for each of the six tables (raw or deduped where applicable)
- [ ] Ensure your team members can access and reproduce your work using the files in the repository.

## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
